# Kelvin-Helmholtz Instability with SPH (Continued)

By now you should have run a simulation and got a reasonable number of snapshots in the output. This notebook will now ingest those outputs and 

## What we'll do in this notebook

1. **Visualize the results** and create movies
2. **Explore different solvers/parameters** to see how they affect the instability growth

Let's get started!

In [7]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py
import unyt
from swiftsimio import load

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. Analysis and Visualization

Let's create some analysis functions that work whether or not you've run the simulation:

In [8]:
# deps: swiftsimio, unyt, matplotlib, numpy, (numba recommended for speed)
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from swiftsimio import load
from swiftsimio.visualisation.projection import project_gas
from unyt import msun, kpc, km, s

def plot_gas_snapshot(filename, ind, title=""):
    """
    Load a SWIFT snapshot (gas only) and make 3 projection panels:
      1) Surface density (Msun/kpc^2)
      2) Mass-weighted <v_x> (km/s)
      3) Mass-weighted <|v|> in xy-plane (km/s)
    Saves to plots/kelvin_helmholtz_{ind}.png
    """
    os.makedirs("plots", exist_ok=True)

    ds = load(filename)
    gas = ds.gas
    time_Gyr = float(ds.metadata.time.to("Gyr"))

    # -------------------------
    # Define mass-weighted fields for projection
    # -------------------------
    # v_x and speed_xy = sqrt(vx^2 + vy^2)
    vx = gas.velocities[:, 0]                     # km/s (unit-aware)
    vy = gas.velocities[:, 1]
    speed_xy = (vx**2 + vy**2)**0.5               # km/s

    gas.mass_weighted_vx = gas.masses * vx        # Msun * (km/s)
    gas.mass_weighted_speed_xy = gas.masses * speed_xy

    # -------------------------
    # Make projections (fast backend by default)
    # -------------------------
    res = 512
    common = dict(data=ds, resolution=res, parallel=True, periodic=True)

    mass_map = project_gas(project="masses", **common)                      # Msun / kpc^2 (cosmo_array)
    mw_vx_map = project_gas(project="mass_weighted_vx", **common)           # Msun*km/s / kpc^2
    mw_speed_map = project_gas(project="mass_weighted_speed_xy", **common)  # Msun*km/s / kpc^2

    # Avoid division issues in empty pixels
    eps = 0.0 * mass_map  # zero in same units/type
    safe_mass = mass_map.copy()
    safe_mass[safe_mass == 0 * (msun / kpc**2)] = eps + 1e-300 * (msun / kpc**2)

    vx_map = (mw_vx_map / safe_mass).to_physical_value(km / s)          # km/s (numpy array)
    speed_map = (mw_speed_map / safe_mass).to_physical_value(km / s)    # km/s

    # Panel 1 needs surface density in physical Msun/kpc^2:
    sigma = mass_map.to_physical_value(msun / kpc**2)                   # numpy array

    # Figure + axes
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Set imshow extent from box size (physical kpc)
    Lx, Ly = ds.metadata.boxsize.to("kpc").value[:2]
    extent = [0.0, Lx, 0.0, Ly]

    # --- Panel 1: surface density ---
    im1 = axes[0].imshow(
        sigma,
        norm=LogNorm(),
        origin="lower",
        extent=extent,
        aspect="equal",
        cmap="viridis",
    )
    axes[0].set_title(f"Surface density (t={time_Gyr:.3f} Gyr)")
    axes[0].set_xlabel("x [kpc]")
    axes[0].set_ylabel("y [kpc]")
    plt.colorbar(im1, ax=axes[0], label=r"$\Sigma_{\rm gas}$ [M$_\odot$/kpc$^2$]")

    # --- Panel 2: mass-weighted <v_x> ---
    im2 = axes[1].imshow(
        vx_map,
        origin="lower",
        extent=extent,
        aspect="equal",
        cmap="RdBu_r",
    )
    axes[1].set_title(r"Mass-weighted $\langle v_x \rangle$")
    axes[1].set_xlabel("x [kpc]")
    axes[1].set_ylabel("y [kpc]")
    plt.colorbar(im2, ax=axes[1], label="km s$^{-1}$")

    # --- Panel 3: mass-weighted <|v|> (xy) ---
    im3 = axes[2].imshow(
        speed_map,
        origin="lower",
        extent=extent,
        aspect="equal",
        cmap="plasma",
    )
    axes[2].set_title(r"Mass-weighted $\langle |v| \rangle$ (xy)")
    axes[2].set_xlabel("x [kpc]")
    axes[2].set_ylabel("y [kpc]")
    plt.colorbar(im3, ax=axes[2], label="km s$^{-1}$")

    if title:
        fig.suptitle(title, fontsize=16)

    plt.tight_layout()
    plt.savefig(f"plots/kelvin_helmholtz_{ind}.png", dpi=150, bbox_inches="tight")
    plt.show()


## 7. Loading and Analyzing Simulation Results

If the simulation ran successfully, let's load and analyze the results:

In [9]:
import glob

# Look for snapshot files
run_name = "GADGET2"
snapshot_files = sorted(glob.glob(f'../runs/{run_name}/kelvin_helmholtz_*.hdf5'))

if snapshot_files:
    print(f"Found {len(snapshot_files)} snapshot files")
    
    for i, snap in enumerate(snapshot_files):
        print(f"\nLoading snapshot {i}: {snap}")
        plot_gas_snapshot(snap, i, title=f"GADGET2: Snapshot {i}")
else:
    print("No snapshot files found. Is the path correct?")

No snapshot files found. Is the path correct?
